In [2]:
from PIL import Image
import numpy as np
import torch
import lpips
import os
from torchvision import transforms

OSError: [WinError 1455] The paging file is too small for this operation to complete. Error loading "c:\Users\Arjit\miniconda3\envs\CT2\lib\site-packages\torch\lib\caffe2_detectron_ops_gpu.dll" or one of its dependencies.

In [ ]:
# Paths to folders containing images
real_images_folder = 'C:/Users/Arjit/OneDrive/Desktop/MTP/CODE/Color Diffusion/Color-diffusion/test_images/100_gt'
fake_images_folder = 'C:/Users/Arjit/OneDrive/Desktop/MTP/CODE/Color Diffusion/Color-diffusion/results/100_output'

In [4]:
# Load images and preprocess them
def load_images_from_folder(folder_path):
    preprocess = transforms.Compose([
        transforms.Resize((256, 256)),  # Resize to 256x256 as LPIPS expects this size
        transforms.ToTensor(),
    ])
    
    images = []
    for filename in os.listdir(folder_path):
        image_path = os.path.join(folder_path, filename)
        image = Image.open(image_path).convert('RGB')
        image_tensor = preprocess(image).unsqueeze(0)  # Add batch dimension
        images.append(image_tensor)
    
    return images

In [5]:
# Initialize LPIPS model
lpips_model = lpips.LPIPS(net='alex')  # You can also use 'vgg' or 'squeeze' as alternatives

Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]


/home/satyanshu/miniconda3/envs/satyan/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/satyanshu/miniconda3/envs/satyan/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /home/satyanshu/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth
100%|██████████| 233M/233M [00:02<00:00, 92.2MB/s] 


Loading model from: /home/satyanshu/miniconda3/envs/satyan/lib/python3.10/site-packages/lpips/weights/v0.1/alex.pth


In [6]:
# Load the model to the same device (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
lpips_model = lpips_model.to(device)

In [7]:
# Load images from both folders
real_images = load_images_from_folder(real_images_folder)
fake_images = load_images_from_folder(fake_images_folder)

In [8]:
# Calculate LPIPS score for each pair of images from both folders
lpips_scores = []
for real_image, fake_image in zip(real_images, fake_images):
    # Ensure both images are on the same device (e.g., CPU or GPU)
    real_image = real_image.to(device)
    fake_image = fake_image.to(device)
    
    # Calculate the LPIPS score
    lpips_score = lpips_model(real_image, fake_image)
    lpips_scores.append(lpips_score.item())  # Convert tensor to scalar

In [9]:
# Calculate the average LPIPS score
average_lpips = np.mean(lpips_scores)
print("Average LPIPS Score:", average_lpips)

Average LPIPS Score: 0.6545513416007162
